# 목차
## 1. Library Import
## 2. Data Set
## 3. Shape Set
## 4. AutoML Set
## 5. Model Train
## 6. Prediction
## 7. Submission

## 1. Library Import

In [1]:
!pip install pycaret

     |████████████████████████████████| 256kB 8.4MB/s 
     |████████████████████████████████| 276kB 14.5MB/s 
     |████████████████████████████████| 65.8MB 55kB/s 
     |████████████████████████████████| 157.5MB 78kB/s 
     |████████████████████████████████| 6.8MB 21.9MB/s 
     |████████████████████████████████| 102kB 18.4MB/s 
     |████████████████████████████████| 1.8MB 55.9MB/s 
     |████████████████████████████████| 14.2MB 16.0MB/s 
     |████████████████████████████████| 174kB 79.1MB/s 
     |████████████████████████████████| 245kB 45.9MB/s 
     |████████████████████████████████| 1.6MB 51.0MB/s 
     |████████████████████████████████| 2.1MB 56.7MB/s 
     |████████████████████████████████| 348kB 56.5MB/s 
     |████████████████████████████████| 61kB 11.6MB/s 
     |████████████████████████████████| 1.1MB 48.0MB/s 
     |████████████████████████████████| 81kB 14.1MB/s 
     |████████████████████████████████| 153kB 82.2MB/s 
     |████████████████████████████████| 163kB 78.0M

In [2]:
import pandas as pd
import numpy as np

import datetime as dt

import matplotlib.pyplot as plt

from tqdm import tqdm

import gc
import random
import lightgbm as lgb
import re

from sklearn.metrics import *
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings(action='ignore')

# 모듈
import os, pathlib, glob
import random
from pytz import timezone # Colab 사용 시, Asia/Seoul로 지정 필요

from pycaret.classification import *

In [3]:
# HDLY
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# HDLY
# 작업경로 변경
os.chdir('/content/gdrive/MyDrive/Colab/Dacon_LG_AUC')

print('Current directory: ', os.getcwd())

Current directory:  /content/gdrive/MyDrive/Colab/Dacon_LG_AUC


In [ ]:
########################################
########################################
########################################

## 2. Data Set

#### 2.1. train_err

In [121]:
PATH = os.getcwd() + '/'

# load
train_err_1 = pd.read_csv(PATH+'HDLY_train_err_with_DayHour.csv')
train_err_1 = train_err_1.iloc[:, :7]

train_err_2 = pd.read_csv(PATH+'HDLY_train_err_with_ErrFwverModelQuality.csv')

train_err_3 = pd.read_csv(PATH+'20210203_train_data_merge.csv')
train_err_3 = train_err_3.iloc[:, -49:-1] # err_hour ~ quality_hour
# train_err_3 = train_err_3.iloc[:, -49:-25] # only err_hour

# concat
train_err = pd.concat([train_err_1, train_err_2], axis=1)
train_err = pd.concat([train_err, train_err_3], axis=1)

# 확인
train_err

,day_0,day_1,day_2,day_3,day_4,day_5,day_6,fw_ctgr_3,fw_ctgr_4,fw_ctgr_5,fw_ctgr_8,fw_ctgr_9,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,...,err_hours_8,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23
0,54.0,52.0,37.0,41.0,45.0,45.0,43.0,0.0,0.0,317.0,0.0,0.0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,...,11.0,8.0,8.0,5.0,5.0,0.0,10.0,9.0,6.0,15.0,19.0,12.0,0.0,10.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
1,117.0,139.0,164.0,138.0,107.0,166.0,1534.0,0.0,2365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,...,29.0,57.0,28.0,83.0,160.0,169.0,183.0,193.0,210.0,153.0,85.0,230.0,176.0,258.0,70.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,42.0,58.0,52.0,34.0,46.0,40.0,34.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,...,10.0,15.0,20.0,8.0,0.0,6.0,4.0,27.0,5.0,6.0,5.0,28.0,24.0,30.0,13.0,10.0,12.0,36.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22.0,57.0,51.0,52.0,42.0,45.0,37.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,...,0.0,4.0,12.0,1.0,3.0,0.0,0.0,8.0,3.0,5.0,5.0,19.0,39.0,30.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,84.0,108.0,96.0,144.0,115.0,59.0,171.0,0.0,777.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,...,30.0,30.0,10.0,8.0,11.0,36.0,8.0,7.0,18.0,22.0,69.0,111.0,43.0,31.0,70.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,39.0,34.0,40.0,0.0,24.0,3.0,54.0,0.0,194.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,...,0.0,0.0,7.0,6.0,0.0,12.0,6.0,0.0,14.0,11.0,11.0,20.0,4.0,14.0,18.0,16.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
14996,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,118.0,133.0,125.0,112.0,133.0,96.0,109.0,0.0,826.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,...,67.0,56.0,9.0,16.0,25.0,14.0,8.0,8.0,11.0,16.0,35.0,19.0,65.0,39.0,62.0,41.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
14998,12.0,24.0,23.0,47.0,22.0,8.0

In [ ]:
########################################
########################################
########################################

#### 2.2. test_err



In [122]:
# load
test_err_1 = pd.read_csv(PATH+'HDLY_test_err_with_DayHour.csv')
test_err_1 = test_err_1.iloc[:, :7]

test_err_2 = pd.read_csv(PATH+'HDLY_test_err_with_ErrFwverModelQuality.csv')

test_err_3 = pd.read_csv(PATH+'20210203_test_data_merge.csv')
test_err_3 = test_err_3.iloc[:, -48:] # err_hour ~ quality_hour
# test_err_3 = test_err_3.iloc[:, -48:-24] # only err_hour

# concat
test_err = pd.concat([test_err_1, test_err_2], axis=1)
test_err = pd.concat([test_err, test_err_3], axis=1)

# 확인
test_err

,day_0,day_1,day_2,day_3,day_4,day_5,day_6,fw_ctgr_3,fw_ctgr_4,fw_ctgr_5,fw_ctgr_8,fw_ctgr_9,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,...,err_hours_8,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23
0,1024.0,510.0,267.0,62.0,410.0,254.0,223.0,0.0,2750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,...,12.0,123.0,30.0,11.0,32.0,31.0,105.0,284.0,157.0,133.0,143.0,522.0,475.0,97.0,113.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
1,54.0,56.0,43.0,32.0,57.0,24.0,18.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,...,18.0,5.0,20.0,12.0,25.0,1.0,9.0,5.0,4.0,14.0,9.0,24.0,12.0,8.0,9.0,19.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
2,140.0,135.0,138.0,120.0,158.0,151.0,99.0,0.0,941.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,...,37.0,26.0,31.0,55.0,26.0,34.0,32.0,47.0,58.0,63.0,92.0,38.0,10.0,23.0,14.0,16.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,24.0,0.0,36.0,0.0,12.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
3,24.0,71.0,49.0,34.0,69.0,32.0,92.0,0.0,371.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,...,14.0,13.0,6.0,4.0,23.0,14.0,6.0,11.0,29.0,13.0,10.0,31.0,13.0,17.0,20.0,10.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0
4,102.0,157.0,109.0,112.0,120.0,176.0,105.0,0.0,881.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,...,99.0,43.0,11.0,11.0,5.0,37.0,66.0,55.0,34.0,32.0,31.0,27.0,32.0,24.0,13.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,116.0,245.0,177.0,133.0,151.0,139.0,154.0,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,...,14.0,13.0,34.0,78.0,68.0,36.0,101.0,37.0,89.0,72.0,97.0,46.0,62.0,128.0,88.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14995,71.0,78.0,83.0,86.0,55.0,68.0,74.0,0.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,12.0,27.0,35.0,46.0,24.0,20.0,12.0,14.0,14.0,29.0,27.0,59.0,18.0,32.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,419.0,533.0,223.0,204.0,169.0,272.0,413.0,0.0,2233.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,...,21.0,28.0,174.0,56.0,110.0,27.0,29.0,113.0,183.0,103.0,103.0,273.0,129.0,164.0,189.0,50.0,12.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.

In [ ]:
########################################
########################################
########################################

#### 2.3. problem

In [123]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 

problem.shape

(15000,)

In [ ]:
########################################
########################################
########################################

## 3. Shape sync.

In [124]:
train_err['problem'] = problem
train_err.drop(['user_id'], axis=1, inplace=True)

In [ ]:
########################################
########################################
########################################

## 4. AutoML

In [125]:
# AutoML

clf2 = setup(data = train_err, target = 'problem') 

,Description,Value
0,session_id,2893
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 125)"
5,Missing Values,False
6,Numeric Features,115
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [126]:
best_3 = compare_models(sort = 'Accuracy', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7914,0.8180,0.5061,0.7962,0.6185,0.4847,0.5084,20.679
gbc,Gradient Boosting Classifier,0.7877,0.8113,0.4813,0.8062,0.6024,0.4700,0.4992,5.637
et,Extra Trees Classifier,0.7875,0.8072,0.5019,0.7851,0.6118,0.4753,0.4981,1.690
lightgbm,Light Gradient Boosting Machine,0.7863,0.8112,0.5107,0.7741,0.6150,0.4756,0.4955,0.704
rf,Random Forest Classifier,0.7789,0.8038,0.4671,0.7855,0.5854,0.4478,0.4761,2.514
xgboost,Extreme Gradient Boosting,0.7753,0.7923,0.5243,0.7280,0.6094,0.4575,0.4698,8.199
ada,Ada Boost Classifier,0.7723,0.7943,0.4828,0.7474,0.5861,0.4390,0.4592,1.205
ridge,Ridge Classifier,0.7660,0.0000,0.3771,0.8308,0.5180,0.3914,0.4448,0.070
lda,Linear Discriminant Analysis,0.7658,0.7680,0.3822,0.8223,0.5211,0.3927,0.4435,0.252
lr,Logistic Regression,0.7352,0.7439,0.3358,0.7269,0.4578,0.3132,0.3547,1.866


In [127]:
# blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')
stacker = stack_models(estimator_list=best_3[1:], meta_model=best_3[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7857,0.8164,0.4729,0.8058,0.5961,0.4634,0.4938
1,0.8019,0.8169,0.5413,0.8017,0.6463,0.5158,0.5350
2,0.7933,0.8202,0.5185,0.7913,0.6265,0.4921,0.5131
3,0.7800,0.8208,0.4758,0.7804,0.5912,0.4525,0.4784
4,0.7971,0.8237,0.5299,0.7949,0.6359,0.5030,0.5228
5,0.7867,0.8111,0.4786,0.8038,0.6000,0.4670,0.4962
6,0.8057,0.8288,0.5299,0.8267,0.6458,0.5206,0.5451
7,0.7829,0.8024,0.4786,0.7887,0.5957,0.4592,0.4860
8,0.8000,0.8327,0.5114,0.8257,0.6316,0.5045,0.5318
9,0.7817,0.7893,0.5071,0.7607,0.6085,0.4655,0.4839


In [128]:
# pred_holdout = predict_model(blended)
pred_holdout = predict_model(stacker)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.7898,0.8114,0.4802,0.8061,0.6019,0.4713,0.5004


In [ ]:
# final_model = finalize_model(blended)
final_model = finalize_model(stacker)

In [ ]:
########################################
########################################
########################################

## 5. Model Train5

In [117]:
# 변수 이름 변경 : error  -> train_x / problem-> train_y
train_err.drop(['problem'], axis=1, inplace=True)
train_x = train_err.to_numpy()
train_y = problem

print(train_x.shape)
print(train_y.shape)

(15000, 100)
(15000,)


In [118]:
# Train

#-------------------------------------------------------------------------------------

# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True

#-------------------------------------------------------------------------------------

models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5

lr = 0.01 #learning_rate
nbr = 20000 #num_boost_round 
vb = 500 #verbose_eval
esr = 1000 #early_stopping_rounds
boosting_type = ['gbdt','dart']

# 파라미터 설정
params =      {
                'boosting_type' : boosting_type[0],
                'objective'     : 'binary',
                'metric'        : 'auc',
                'learning_rate' : lr,
                'seed': 2021
                }

es_only_dart = [8000, 12000, 5500, 10500]
es_only_index = 0

#-------------------------------------------------------------------------------------

# 5 Kfold cross validation
k_fold = KFold(n_splits=4, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        # num_boost_round = nbr,
                        num_boost_round = es_only_dart[es_only_index],
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = vb, 
                        early_stopping_rounds = esr
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')
    es_only_index += 1


print(f"RESULT AUC : {np.mean(auc_scores)}")

[LightGBM] [Info] Number of positive: 3610, number of negative: 7640
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004973 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14701
[LightGBM] [Info] Number of data points in the train set: 11250, number of used features: 96
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.320889 -> initscore=-0.749690
[LightGBM] [Info] Start training from score -0.749690
Training until validation scores don't improve for 1000 rounds
[500]	valid_0's auc: 0.807665	valid_0's pr_auc: 0.753344
[1000]	valid_0's auc: 0.80788	valid_0's pr_auc: 0.753814
[1500]	valid_0's auc: 0.8077	valid_0's pr_auc: 0.752071
Early stopping, best iteration is:
[634]	valid_0's auc: 0.808776	valid_0's pr_auc: 0.755438
[LightGBM] [Info] Number of positive: 4775, number of negative: 6475
[LightGBM] [Warning] Auto-choosing row-wise mu

In [ ]:
########################################
########################################
########################################

## 6. Prediction

In [119]:
# 예측

t_e_corr = train_err.corr()
t_list = list(t_e_corr.index)

pred_y_list = []

for model in models:
    pred_y = model.predict(test_err[t_list].to_numpy())
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

pred_ensemble

array([[0.9208245 ],
       [0.21576706],
       [0.25510672],
       ...,
       [0.45579213],
       [0.89580054],
       [0.388417  ]])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(250,250))
sns.heatmap(data = train_err.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
########################################
########################################
########################################

## 7. submission to_csv

In [120]:
# HDLY

# 제출

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = pred_ensemble.reshape(-1)

sample_submssion.to_csv("HDLY_0203_4_dart_7cols_08139255425947884.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.920825
1,30001,0.215767
2,30002,0.255107
3,30003,0.716601
4,30004,0.657507
...,...,...
14994,44994,0.306050
14995,44995,0.330715
14996,44996,0.455792
14997,44997,0.895801


In [ ]:
########################################
########################################
########################################